## Method to extract required infomation from email files

In [1]:
import pandas as pd
import numpy as np
import email
import os
from email.parser import Parser
import re

In [2]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [3]:
# extract the information of email
# still need some works to get number of cc and bcc
def info_of_email(path,List_of_files,classification):
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    parser = Parser()
    info = []
    for files in List_of_files:
        emailText =  open(path + files, 'rb').read()
        email = parser.parsestr(emailText)
        if email.get('cc') == None:
            no_cc = 0
        else:
            no_cc = len(email.get('cc'))
        if email.get("bcc") == None:
            no_bcc = 0
        else:
            no_bcc = len(email.get('bcc'))
        Date = email.get("Date")
        special_chars = re.compile(r"\t\n\r")
        try:
            Date = re.split(':|,| ',Date)
            info.append([Date[4], Date[3], Date[2], Date[0], Date[5], email.get("From"), 
                         email.get('To'), no_cc, no_bcc, 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), 
                         " ".join([i for i in walkMsg(email) if 
                                   i != None]).replace("\n"," ").replace("\t"," ").replace("r"," "),
                         classification])
        except:
            info.append([[], [], [], [], [], email.get("From"), 
                         email.get('To'), no_cc, no_bcc, 
                         str(email.get("Content-Type")).split(";")[0],
                         email.get('subject'), 
                         " ".join([i for i in walkMsg(email) if 
                                   i != None]).replace("\n"," ").replace("\t"," ").replace("r"," "),
                         classification])
    return info

### Spam

In [4]:
listing = os.listdir('./Spam/')
listing = [i for i in listing if i[-5:] != "Store"]
email_df = pd.DataFrame(info_of_email('./Spam/', listing, 'spam'),
                       columns=['Year','Month','Day','Weekday',"Hour",
                                'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [5]:
email_df.tail()

,Year,Month,Day,Weekday,Hour,From,To,cc,bcc,Content_type,Subjects,Content,Classification
31096,2002,Dec,3,Tue,18,"""IQ - TBA"" <tba@insiq.us>",<yyyy@spamassassin.taint.org>,0,0,multipart/alternative,Preferred Non-Smoker Rates for Smokers,P efe ed Non-Smoke Just what the doct...,spam
31097,2003,Jul,20,Sun,16,Mike <raye@yahoo.lv>,Mailing.List@user2.pro-ns.net,0,0,text/plain,"How to get 10,000 FREE hits per day to any web...","Dea Subsc ibe , If I could show you a way to...",spam
31098,2020,Aug,05,Wed,04,"""Mr. Clean"" <cweqx@dialix.oz.au>",<Undisclosed.Recipients@webnote.net>,0,0,text/plain,Cannabis Difference,****Mid-Summe Custome App eciation SALE!****...,spam
31099,2002,Dec,04,Wed,06,"""wilsonkamela400@netscape.net"" <wilsonkamela50...",ilug@linux.ie,0,0,text/plain,[ILUG] WILSON KAMELA,ATTN:SIR/MADAN ST...,spam
31100,2005,Jul,05,Tue,09,"""Chia Patterson"" <pkrebehenne@businessesinport...",projecthoneypot@projecthoneypot.org,0,0,text/html,Just to her...,"<html> <body alink=""blue"" bgcolo =""white"" link...",spam


In [6]:
email_df.shape

(31101, 13)

### Ham

In [7]:
path = './Ham/'
direc = os.listdir(path)
direc = [i for i in direc if i[-5:] != "Store"]

In [8]:
file_name = []
for i in direc:
    file_name.extend([path + i + "/" + f for f in os.listdir(path+i)])

In [9]:
ham_df = pd.DataFrame(info_of_email("",file_name,"ham"),
                      columns=['Year','Month','Day','Weekday',"Hour",
                                'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [10]:
ham_df.head()

,Year,Month,Day,Weekday,Hour,From,To,cc,bcc,Content_type,Subjects,Content,Classification
0,2001,Dec,2,Sun,12,no.address@enron.com,jeff.donahue@enron.com,24,24,text/plain,Asset Marketing/Corporate Development,"Jeff, As we discussed, attached is a comp ehe...",ham
1,2001,Sep,27,Thu,11,no.address@enron.com,"m..presto@enron.com, mitch.robinson@enron.com,...",72,72,text/plain,RE: Conn. Coal Plants,Ou esea ch indicates that in 1998 Wisvest pa...,ham
2,2001,Mar,5,Mon,23,don.miller@enron.com,louise.kitchen@enron.com,0,0,text/plain,Peaker Update,"Louise, At you ea liest convenience, I need ...",ham
3,2001,Mar,2,Fri,18,don.miller@enron.com,louise.kitchen@enron.com,0,0,text/plain,Revised 1999 and 2000 Peaker Action Lists as o...,"Louise, Attached a e the Cine gy and Alleghen...",ham
4,2001,Sep,26,Wed,12,no.address@enron.com,louise.kitchen@enron.com,0,0,text/plain,2001 Gain Calculation,"Louise, Pe you equest. Let me know if you...",ham


In [11]:
ham_df.shape

(18228, 13)

In [12]:
enron_email = email_df.append(ham_df)

### Output the data as pickle file

I choose to output the data in pickle because if output .csv file, we will face difficult when reading it with pd.read_csv

In [13]:
enron_email.to_pickle("enron_email.txt")

In [15]:
import pickle
enron_pickle = pickle.load(open("enron_email.txt","rb"))

In [16]:
enron_pickle.head()

,Year,Month,Day,Weekday,Hour,From,To,cc,bcc,Content_type,Subjects,Content,Classification
0,2004,Jul,31,Sat,12,"""Bertha "" <Denny@mailbox.sk>",rait@bruce-guenter.dyndns.org,0,0,multipart/alternative,Get it up and keep it up praecox,<html><head><meta http-equiv=Content-Type cont...,spam
1,2004,Jul,31,Sat,22,"""LZEDEMCNODQBEZQBOV@encryption.com"" <Opal.Cham...",cvs@bruce-guenter.dyndns.org,29,0,multipart/alternative,Winner: $332573,<html> You [m]o tgage application was app ove...,spam
2,2004,Aug,01,Sun,01,Connie Wong <tatypin@huhmail.com>,Rait <rait@bruce-guenter.dyndns.org>,0,0,multipart/alternative,Need software? Click here.,TOP quality softwa e:<b ><b > <b>Special Offe ...,spam
3,2004,Jul,30,Fri,17,shark@promotions.com <shark@promotions.com>,"""bruceg@em.ca"" <bruceg@em.ca>",0,0,multipart/alternative,Loyalty Bonuses for Shark Casino Players!,Featu ing this weekend 2...,spam
4,2004,Jul,31,Sat,06,"""Important! Search Engine Alert!"" <rockwelldat...","""Bruceg"" <bruceg@em.ca>",0,0,multipart/alternative,Your web site has dropped out of the Search En...,Rockwell Data Co p This Email has been sent ...,spam
